# Warm Thermometry Readout

take note of the following: 
- you need the following csv file in your current working directory: 'warm_thermometry_resistance_temperature_curve.csv'

- this script will create a 'data' folder in your current working directory and save your data to it via csv files

Import statements

In [1]:
import os
import sys
import csv
import numpy as np
import pandas as pd
from labjack import ljm
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import date, datetime

Open up labjack

In [3]:
# open first found LabJack and call it handle
handle = ljm.openS("ANY", "ANY", "ANY")  # Any device, Any connection, Any identifier

# grab and print out important info 
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

Opened a LabJack with Device type: 7, Connection type: 1,
Serial number: 470024022, IP address: 0.0.0.0, Port: 0,
Max bytes per MB: 64


Configure the following parameters:
- which warm thermometry box you're using
- number of readout loops
- bias voltage
- constant resistor value
- resistance to temperature curve
- analog input channels
- labjack readin range, resolution, and settling time
- scan rate
- interval between readings

In [4]:
# determines which warm thermometry box is being used
try:
    box = int(input("Enter '0' if you are using the elevation structure thermometry box,\nEnter '1' if you are using the IS/Yoke thermometry box: \n"))
except Exception:
    print(sys.exc_info()[1])
    raise

# loop_amount determines how many readout loops the labjack will perform 
try:
    loop_amount = input("Enter the number of readout loops you would like performed (type either an integer or the word 'infinite'): \n")
    if loop_amount != "infinite": loop_amount = int(loop_amount)
except Exception:
    print(sys.exc_info()[1])
    raise
    
# bias config
try:
    bias = float(input("Enter your DC bias level (in volts):  \n"))
except Exception:
    print(sys.exc_info()[1])
    raise

# constant resistor config, should be 10kohms
r_constant = 10000

# resistance to temperature config
scriptDir = os.path.dirname(os.path.realpath("__file__"))
file = scriptDir + os.path.sep + "warm_thermometry_resistance_temperature_curve.csv"
temp_list, res_list = np.loadtxt(file, delimiter = ",", skiprows = 0, usecols = (0, 1), unpack = True)
temp_list = temp_list[::-1]
res_list = res_list[::-1]

# analog input channels to be read out
elevation_channels = np.append(np.arange(48,72), np.arange(96,110))
azimuthal_channels = np.arange(98,106)
if box == 0: 
    channel_numbers = elevation_channels 
    box_name = 'elevation'
else: 
    channel_numbers = azimuthal_channels
    box_name = 'azimuthal'


# set range, resolution, and settling time for channels
# note:
#   Negative channel: single ended = 199, differential = 1
#   Range: +/-10.0V (10.0)
#   Resolution index = Default (0)
#   Settling, in microseconds = Auto (0) resource on settling times: https://old3.labjack.com/support/app-notes/SettlingTime
setup_names = ["AIN_ALL_NEGATIVE_CH", "AIN_ALL_RANGE","AIN_ALL_RESOLUTION_INDEX","AIN_ALL_SETTLING_US"]
setup_values = [199,10.0,0,0]
channel_names = []
for c in channel_numbers: channel_names = channel_names+["AIN%d"%c]

# assign the values of range, resolution, and settling to each channel
ljm.eWriteNames(handle, len(setup_names), setup_names, setup_values)

# list of channels to scan
scan_list = ljm.namesToAddresses(len(channel_names), channel_names)[0]

# set scan rate, in Hz
scan_rate = float(input("Enter your desired scan rate, per channel, in Hz (e.g. I want to sample each channel x number of times per second): \n"))

# interval between readings, in microseconds
interval = int((1/scan_rate)*1e6)

# print to see if channels were set up properly
print("\nSet configuration:")
print("    Thermometry box: %s" %box_name)
print("    Number of readout loops: %s" %loop_amount)
print("    Bias amplitude: %sV" %bias)
print("    Scan rate: %sHz" %scan_rate)
print("    Minimum interval between scans of individual channels: %ss" %interval)
for i in range(len(setup_names)):
    print("    %s : %f" % (setup_names[i], setup_values[i]))

Enter '0' if you are using the elevation structure thermometry box,
Enter '1' if you are using the IS/Yoke thermometry box: 
0
Enter the number of readout loops you would like performed (type either an integer or the word 'infinite'): 
infinite
Enter your DC bias level (in volts):  
1
Enter your desired scan rate, per channel, in Hz (e.g. I want to sample each channel x number of times per second): 
1000

Set configuration:
    Thermometry box: elevation
    Number of readout loops: infinite
    Bias amplitude: 1.0V
    Scan rate: 1000.0Hz
    Minimum interval between scans of individual channels: 1000s
    AIN_ALL_NEGATIVE_CH : 199.000000
    AIN_ALL_RANGE : 10.000000
    AIN_ALL_RESOLUTION_INDEX : 0.000000
    AIN_ALL_SETTLING_US : 0.000000


The following cell will do all the data collection and store collected data in a dictionary:

In [5]:
interval_handle = 1
ljm.startInterval(interval_handle, interval)  # Delay between readings (in microseconds)

dictionary = {}
for n in channel_names:
    dictionary[n] = {"V [V]":[],"R [omhs]":[],"Temp [C]":[],"Temp [F]":[],"Temp [K]":[],"Time":[]}

#just a little message
loop_message = " Press Ctrl+C to stop."
print("\nStarting %s read loops.%s\n" % (str(loop_amount), loop_message))
    

i = 0
while True:
    try:
        v_measured = ljm.eReadNames(handle, len(channel_names), channel_names)
        time = datetime.now().strftime("%Y/%m/%d, %H:%M:%S")
        for j in range(len(channel_names)):
            dictionary[channel_names[j]]["V [V]"].append(v_measured[j])
            dictionary[channel_names[j]]["Time"].append(time)
                
        ljm.waitForNextInterval(interval_handle)
        
        if loop_amount != "infinite":
            i = i + 1
            if i >= loop_amount:
                break
    except KeyboardInterrupt:
        break
    except Exception:
        print(sys.exc_info()[1])
        break

# Close handles
ljm.cleanInterval(interval_handle)
ljm.close(handle)


Starting infinite read loops. Press Ctrl+C to stop.



The following cell takes all the collected voltage data, converts it to resistance and temperature values, makes a 'data' folder in your current working directory, and saves all your collected data to csv files:

In [6]:
# for n in channel_names:
#     for i in range(len(dictionary[n]['V [V]'])):
#         dictionary[n]['V [V]'][i] = abs(dictionary[n]['V [V]'][i])

In [7]:
# j = 0
# for i in range(len(v_measured)):
#     dictionary[channel_names[j]]["V [V]"].append(v_measured[i])
#     dictionary[channel_names[j]]["Time"].append(time[i])
#     j+=1
#     if j == len(channel_names): j = 0
    
for n in channel_names:
    for l in range(len(dictionary[n]["V [V]"])):
        r_var = (bias - dictionary[n]["V [V]"][l])*r_constant / dictionary[n]["V [V]"][l]
        dictionary[n]["R [omhs]"].append(r_var)
        temp_C = (np.interp(r_var,res_list,temp_list))
        dictionary[n]["Temp [C]"].append(temp_C)
        temp_F = ((temp_C*(9/5)) + 32)
        dictionary[n]["Temp [F]"].append(temp_F)
        temp_K = (temp_C + 273.15)
        dictionary[n]["Temp [K]"].append(temp_K)
        

# csv data files
if os.path.exists('data/%s_data' %(datetime.now().strftime("%Y-%m-%d"))) == False:
    os.mkdir('data/%s_data' %(datetime.now().strftime("%Y-%m-%d")))

files = []
for i in range(len(channel_names)):
    files.append('data/%s_data' %(datetime.now().strftime("%Y-%m-%d")) + "/thermometer_%s" %channel_names[i])

for n in files:
    df = pd.DataFrame(dictionary[n[33:]])
    df.to_csv(n, index = False)

The following three cells open up your saved data and plot them

In [9]:
print(files[0])
df = pd.read_csv(files[0])
df.head()

data/2023-05-09_data/thermometer_AIN48


,V [V],R [omhs],Temp [C],Temp [F],Temp [K],Time
0,0.000014,7.378671e+08,-40.0,-40.0,233.15,"2023/05/09, 15:57:03"
1,-10.569863,-1.094609e+04,125.0,257.0,398.15,"2023/05/09, 15:57:03"
2,-10.573771,-1.094574e+04,125.0,257.0,398.15,"2023/05/09, 15:57:04"
3,-10.573757,-1.094574e+04,125.0,257.0,398.15,"2023/05/09, 15:57:04"
4,-10.573767,-1.094574e+04,125.0,257.0,398.15,"2023/05/09, 15:57:04"


In [52]:
print(files[17])
df = pd.read_csv(files[17])
df.head()

data/2023-04-17_data/thermometer_AIN65


,V [V],R [omhs],Temp [C],Temp [F],Temp [K],Time
0,7.966657,-6861.920754,125.0,257.0,398.15,"2023/04/17, 14:08:52"
1,7.950618,-6855.590262,125.0,257.0,398.15,"2023/04/17, 14:08:53"
2,7.903607,-6836.887216,125.0,257.0,398.15,"2023/04/17, 14:08:54"
3,7.865761,-6821.667907,125.0,257.0,398.15,"2023/04/17, 14:08:55"
4,7.834947,-6809.167994,125.0,257.0,398.15,"2023/04/17, 14:08:56"


In [53]:
figR = px.line(df, x = 'Time', y = 'R [omhs]', title='Resistance over time')
figR.show()

figF = px.line(df, x = 'Time', y = 'Temp [F]', title='Temperature [F] over time')
figF.show()

figC = px.line(df, x = 'Time', y = 'Temp [C]', title='Temperature [C] over time')
figC.show()

figK = px.line(df, x = 'Time', y = 'Temp [K]', title='Temperature [K] over time')
figK.show()

Quick demonstration of your temperature vs. resistance curve

In [12]:
px.line(x=res_list,y=temp_list)